In [1]:
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import medfilt
import os
import csv
from datetime import datetime
from scipy.signal import argrelextrema
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import os
os.chdir("/content/drive/MyDrive/My_Dissertation/")

In [4]:
filename = "test3_output2.csv"
initial_personality_means = [0.5]*5
weight = 0.5
alpha = 0
valence_weights = [0.5,0.5,0.1]
arousal_weights = [0.8,0.2,0.1]
ow=np.array([[0.33, 0, 0.66, 0, 0, 0, 0, 0,0,0],   #o_def
    [0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0,0,0],           #c_def
    [0.0, 0, 0.33, 0, 0, 0, 0.66, 0,0,0],           #e_def
    [0.0, 0, 0, 0, 0.66, 0, 0.33, 0,0,0],            #a_def
    [0, 0.25, 0, 0.25, 0, 0.25, 0, 0.25,0,0]])     

In [5]:
data_2_x = np.linspace(-1,1,7782)
data_2_y = data_2_x

In [6]:
def read_csv(filename):
        MFC_data = pd.read_csv(filename)
        time = MFC_data['time'].to_numpy()
        data = MFC_data.loc[:,'cell_1':].to_numpy()
        return time,data

In [7]:
class Robot_personality:
    def __init__(self,mu,ow,weight=0.5,alpha = 0.5):
        self.personality = {}
        self.create_random_personality(mu)
        self.initial_personality = self.personality.copy()
        #print("hi")
        self.delta = {}
        self.delta_mean = {}
        self.valence = 0.0
        self.arousal = 0.0
        self.likeness = 0.0
        self.emotion_bins = np.linspace(0,2*math.pi,13)
        self.emotions = {0:"pleased",1:"happy",2:"excited",3:"annoying",4:"angry",5:"nervous",6:"sad",7:"bored",8:"sleepy",9:"calm",10:"peaceful",11:"relaxed"}
        self.emotions_marker = {0:"lightgreen",1:"yellow",2:"gold",3:"darkorange",4:"darksalmon",5:"red",6:'purple',7:"blueviolet",8:"blue",9:"dodgerblue",10:"cyan",11:"lime"}
        self.emotion_val = 0
        self.mfc_data = np.empty((0,1),"float32")
        self.mfc_data_norm = np.empty((0,1),"float32")
        self.time_data = np.empty((0,1),"float32")
        self.stability_bins = [0]*4 #se,sg,sr,sb
        self.valence_bins = [0]*4
        
        self.order = 0
        self.answer = 0
        self.mean_mfc = 0.0
        self.mfc_min = 80
        self.mfc_max = 300
        self.S = 0.001
        self.W = weight
        self.alpha = 0.1
        self.start_smooth = 1
        self.delta_evolution = []
        self.personality_evolution = []
        self.magnitude_evolution = []
        self.phi_evolution = []
        self.bin_evolution = []
        self.valence_evolution = []
        self.arousal_evolution = []
    def create_random_personality(self,mu):
        sigma = 0.001
        keys = "ocean"
        if len(mu)==5:
            personality = [np.random.normal(i,sigma) for i in mu]
            self.personality  = {keys[i]:personality[i] for i in range(len(keys))}
        else:
            print("error creating personalities....setting default personality ")
            personality = [0.5*5]
        return personality

    def update_order_answer_likeness(self,o=0,a=0,l=0):
        self.order = (o+1)/2
        self.answer = (a+1)/2
        self.likeness = l#math.sin(self.time_data[-1])

    ##### add to to object memory  #####
    def update_mfc_data(self,data,time):

        self.mfc_data = np.append(self.mfc_data,data)
        self.time_data = np.append(self.time_data,time)
        self.mfc_data_norm = 2*((self.mfc_data-self.mfc_min)/(self.mfc_max-self.mfc_min))-1
        
        


        #self.diff = np.abs(self.diff-np.mean(self.diff))/np.std(self.diff)
        #self.diff = (np.clip(self.diff,-2,2)/2.5)


    ##### update valence and arousal #####
    def update_mood(self,valence_weights,arousal_weights):
        self.valence = (((valence_weights[0]*(self.mfc_data_norm[-1]+1)/2+valence_weights[1]*(self.likeness+1)/2)*(1-valence_weights[2]*self.personality["n"]))*2)-1
        self.arousal = (((arousal_weights[0]*(self.mfc_data_norm[-1]+1)/2+arousal_weights[1]*(self.likeness+1)/2)*(1-arousal_weights[2]*self.personality["n"]))*2)-1
        self.valence_evolution.append(self.valence)
        self.arousal_evolution.append(self.arousal)
        mean = np.mean(self.valence_evolution[:-1])
        sigma = np.std(self.valence_evolution[:-1])
        if len(self.valence_evolution)>0:
          if self.valence_evolution[-1] < mean-sigma :
            self.valence_bins[3]+=1
          elif self.valence_evolution[-1] >= mean-sigma and self.valence_evolution[-1] <= mean:
            self.valence_bins[2]+=1
          elif self.valence_evolution[-1] > mean and self.valence_evolution[-1] <= mean+sigma:
            self.valence_bins[1]+=1
          else:
            self.valence_bins[0]+=1
          self.happiness = (2*self.valence_bins[0] +self.valence_bins[1])/(2*self.valence_bins[0] +self.valence_bins[1]+2*self.valence_bins[3] +self.stability_bins[2])
          #print(self.valence_bins)
        
        return self.valence,self.arousal,self.happiness

    ##### update delta  #########
    def update_o(self):
        self.delta["o"] = (1/3)*self.happiness + (2/3)*self.answer
        
        return self.delta["o"]

    def update_c(self):
        self.diff = np.abs(np.diff(self.arousal_evolution))
        mean = np.mean(self.diff[:-1])
        sigma = np.std(self.diff[:-1])
        if len(self.diff)>0:
          if self.diff[-1] < mean-sigma :
            self.stability_bins[0]+=1
          elif self.diff[-1] >= mean-sigma and self.diff[-1] <= mean:
            self.stability_bins[1]+=1
          elif self.diff[-1] > mean and self.diff[-1] <= mean+sigma:
            self.stability_bins[2]+=1
          else:
            self.stability_bins[3]+=1

        
        if len(self.diff)<=0:
          self.delta["c"] = self.initial_personality["c"]
        else:
          self.delta["c"] = (2*self.stability_bins[0] +self.stability_bins[1])/(2*self.stability_bins[0] +self.stability_bins[1]+2*self.stability_bins[3] +self.stability_bins[2])

        if self.delta["c"]==0:
          self.delta["c"] = self.initial_personality["c"]

        return self.delta["c"]
    def update_e(self):
        self.delta["e"] = (2/3)*self.happiness + (1/3)*self.answer
        
        return self.delta["e"]
    def update_a(self):
        self.delta["a"] = (1/3)*self.happiness + (2/3)*(1-self.order)
        
        return self.delta["a"]
    def update_n(self):
        self.delta["n"] = (1/2)*(1-self.happiness) + (1/4)*(1-self.order) + (1/4)*(1-self.answer)
        #print(self.delta["n"])
        #self.delta["n"] = (self.delta["n"]+1)/2
        return self.delta["n"]
    def update_delta_evolution(self):
        self.update_o()
        self.update_c()
        self.update_e()
        self.update_a()
        self.update_n()
        self.delta_evolution.append(np.array(list(self.delta.values()),dtype=float))
        self.delta_mean = np.mean(self.delta_evolution,axis=0)
        keys = "ocean"
        self.delta_mean  = {keys[i]:self.delta_mean[i] for i in range(len(keys))}



    ##### update personality #########
    def smoothing(self):

      self.S = self.start_smooth*(1-math.tanh(self.alpha*self.time_data[-1]))
      self.S=1
      for i in self.personality.keys():
        self.personality[i] =  self.S*self.personality[i] + (1-self.S)*self.prev_personality[i]
      #print(self.S)
    def update_personality(self):
        
        self.prev_personality = self.personality.copy()
        for i in self.personality.keys():
          self.personality[i] = self.W*self.initial_personality[i] + (1-self.W)*(self.delta[i])
        self.smoothing()  
        self.personality_evolution.append(np.array(list(self.personality.values()),dtype=float))
        return np.array(list(self.personality.values()),dtype=float)

    ##### update emotions
    def bin(self):
        for i in range(12):
          if self.phi>self.emotion_bins[i] and self.phi<=self.emotion_bins[i+1]:
            
            self.emotion_val = i
            return i
        else:
          print("none")
    def emotion_gen(self):
        self.phi = math.atan2(self.arousal,self.valence)
        if self.phi<0:
          self.phi = math.pi + (math.pi+self.phi)
        self.magnitude = math.sqrt(math.pow(self.arousal,2)+math.pow(self.valence,2))
        self.phi_evolution.append(self.phi)
        self.magnitude_evolution.append(self.magnitude)
        self.bin_evolution.append(self.bin())
        
        

In [8]:
#time,data = read_csv(filename)
with open('./mfc_data.npy', 'rb') as f:
    time = np.load(f)
    data = np.load(f)
#mfc_2 = data[:3891]
#time = time[:3891]
robot_1 = Robot_personality(initial_personality_means,ow,weight,alpha)
order = -0.5
answer = 0.5
likeness = np.sin(np.linspace(0,10,len(time)))
valence = []
arousal = []
happiness = []
op = []
co = []
ex =[]
ag =[]
ne = []
personality_evolution=[]

In [9]:
robot_1.personality

{'o': 0.5005598493804332,
 'c': 0.5001489678633527,
 'e': 0.500681088944206,
 'a': 0.4999366872108072,
 'n': 0.5002881857045032}

In [ ]:
for i in range(len(time)):
    robot_1.update_mfc_data(data[i,1],time[i])
    #robot_2.update_mfc_data(data[i,2],time[i])
    #robot_3.update_mfc_data(data[i,1],time[i])
    if i>0:
        robot_1.update_order_answer_likeness(order,answer,likeness[i])
        val,aro,happ = robot_1.update_mood(valence_weights,arousal_weights)
        valence.append(val)
        arousal.append(aro)
        happiness.append(happ)
        robot_1.update_delta_evolution()
        robot_1.update_personality()
        robot_1.emotion_gen()



/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
robot_1.personality_evolution = np.stack(robot_1.personality_evolution,axis=0)


In [ ]:
robot_1.personality_evolution.shape

In [ ]:
robot_1.delta_evolution = np.stack(robot_1.delta_evolution,axis=0)


#Cell 1

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(happiness)
ax.set_xlabel("Time (days)",fontsize=15)
ax.set_ylabel("MFC output normalized",fontsize=15)
plt.savefig("./MFC graphs/Cell_3_2wks_norm.png")

In [ ]:
with open('./times_2.npy', 'wb') as f:
  np.save(f, time)
  #np.save(f,robot_1.personality_evolution)

In [ ]:
#with open('./gen2_cell_3.npy', 'wb') as f:
#  np.save(f, robot_1.delta_evolution)
#  np.save(f,robot_1.personality_evolution)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(time[10:-1],robot_1.delta_evolution[10:,0],label = "openess")
ax.plot(time[10:-1],robot_1.delta_evolution[10:,1],label = "Conscientioussness")
ax.plot(time[10:-1],robot_1.delta_evolution[10:,2],label = "extraversion")
ax.plot(time[10:-1],robot_1.delta_evolution[10:,3],label = "agreeableness")
ax.plot(time[10:-1],robot_1.delta_evolution[10:,4],label = "neuroticism")
ax.set_xlabel("Time (days)",fontsize=15)
ax.set_ylabel("Score",fontsize=15)
ax.legend()
plt.ylim([0,1])
plt.savefig("./Experiment_3/acquired_Cell_1_.png")

In [ ]:
robot_1.stability_bins

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(time[:-1],robot_1.personality_evolution[:,0],label = "openess")
ax.plot(time[:-1],robot_1.personality_evolution[:,1],label = "Conscientioussness")
ax.plot(time[:-1],robot_1.personality_evolution[:,2],label = "extraversion")
ax.plot(time[:-1],robot_1.personality_evolution[:,3],label = "agreeableness")
ax.plot(time[:-1],robot_1.personality_evolution[:,4],label = "neuroticism")
ax.set_xlabel("Time (days)",fontsize=15)
ax.set_ylabel("Score",fontsize=15)
ax.legend()
plt.ylim([0,1])
#plt.savefig("./Experiment_3/Final_Cell_2.png")

In [ ]:
import matplotlib.animation as animation

In [ ]:

fig, ax = plt.subplots(figsize=(8,6))
ax = plt.gca()
def animate(i):

    ax.clear()
    
    ax.spines['top'].set_color('none')
    ax.spines['left'].set_position('zero')
    ax.spines['right'].set_color('none')
    ax.spines['bottom'].set_position('zero')
    ax.scatter(valence[i], arousal[i],c = robot_1.emotions_marker[robot_1.bin_evolution[i]],label = robot_1.emotions[robot_1.bin_evolution[i]])
    #print(i)
    ax.set_xlabel("Valence",fontsize=15,ha="right",position=(1,0))
    
    ax.set_ylabel("Arousal",fontsize=15,verticalalignment="top",position=(-.9,.9))
    #ax.legend()
    plt.xlim([-1,1])
    plt.ylim([-1,1])
    ax.legend()
ani = animation.FuncAnimation(fig, animate, frames=7000)
ani.save('./Emotion_gen.mp4', writer = "ffmpeg", fps=100 )
plt.show()


In [ ]:
robot_1.diff.shape

In [ ]:

fig, ax = plt.subplots(figsize=(8,6))
ax = plt.gca()
ax.spines['top'].set_color('none')
ax.spines['left'].set_position('zero')
ax.spines['right'].set_color('none')
ax.spines['bottom'].set_position('zero')
ax.scatter(valence, arousal,label = robot_1.emotions[robot_1.bin_evolution[-1]])
ax.set_xlabel("Time (min)",fontsize=15)
ax.set_ylabel("MFC output (V)",fontsize=15)

plt.xlim([-1,1])
plt.ylim([-1,1])
ax.legend()

In [ ]:
alph = 1
x = np.linspace(0,28,7781)
y = [(1-math.tanh(alph*i)) for i in x]
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(x,y,label = "openess")
ax.set_xlabel("Time (days)",fontsize=15)
ax.set_ylabel("St",fontsize=15)
#plt.savefig("./Experiment_3/st_alpha1.png")